In [ ]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import tmp.set_env

In [ ]:
from load_dataset import EXTRACT_DIR, COL_NAME_IMAGE_FILE, COL_NAME_NAME, COL_NAME_DESCRIPTION, CLIP_TEXTUAL_EMBS_DIR, \
    COL_NAME_ITEM_ID, COL_NAME_IMAGE_ID, DATASET_DIR
from scann.scann_ops.py import scann_ops_pybind
from scann.scann_ops.py.scann_ops import ScannSearcher
from load_dataset import get_dataframe
from load_dataset import CLIP_VISUAL_EMBS_DIR
from utils import load_embeddings, public_dataset_to_html, chunker
import numpy as np
import pandas as pd
from tqdm import tqdm
from load_dataset import COL_NAME_IMAGE_URL
from utils import image_formatter
from IPython.core.display import HTML



In [ ]:

print(EXTRACT_DIR + 'glami-1m-train-index')
index: ScannSearcher = scann_ops_pybind.load_searcher(EXTRACT_DIR + 'glami-1m-train-index')


In [ ]:

test_df = get_dataframe("test")
train_df = get_dataframe("train")


In [ ]:

test_embeddings_array = load_embeddings(test_df, CLIP_VISUAL_EMBS_DIR)
np.save(EXTRACT_DIR + 'test_embeddings_array.npy', test_embeddings_array)


In [ ]:
test_embeddings_array = np.load(EXTRACT_DIR + 'test_embeddings_array.npy')

In [ ]:
idxs, sims = index.search(test_embeddings_array[0], final_num_neighbors=3)


In [ ]:
public_dataset_to_html(test_df.loc[[0]])

In [ ]:

# TODO render image
public_dataset_to_html(train_df.loc[idxs])


In [ ]:
batch_size = 1024
n_neighbors = 5
closest_train_indexes = np.zeros((len(test_df), n_neighbors), dtype=int)
closest_sims = np.zeros((len(test_df), n_neighbors), dtype=float)

test_df['index'] = np.arange(len(test_df))
n = int(np.ceil(len(test_df) / batch_size))
for i in tqdm(test_df['index'].values):
    lower = i * batch_size
    upper = min((i + 1) * batch_size, len(test_df))
    q = test_embeddings_array[lower:upper]
    closest_train_indexes[lower:upper], closest_sims[lower:upper] = index.search_batched_parallel(q, n_neighbors)


np.save(EXTRACT_DIR + 'closest_train_indexes.npy', closest_train_indexes)
np.save(EXTRACT_DIR + 'closest_sims.npy', closest_sims)


In [ ]:
closest_train_indexes= np.load(EXTRACT_DIR + 'closest_train_indexes.npy')
closest_sims = np.load(EXTRACT_DIR + 'closest_sims.npy')


In [ ]:
SIM_PREFIX = 'sim_'
test_df['index'] = np.arange(len(test_df))
test_df['top_1_train_index'] = closest_train_indexes[:, 0]
test_df['top_1_sim'] = closest_sims[:, 0]
test_df[[SIM_PREFIX + COL_NAME_IMAGE_FILE, SIM_PREFIX + COL_NAME_NAME, SIM_PREFIX + COL_NAME_DESCRIPTION]] = test_df['top_1_train_index'].apply(lambda i: train_df.iloc[i][[COL_NAME_IMAGE_FILE, COL_NAME_NAME, COL_NAME_DESCRIPTION]])
sorted_test_df = test_df.sort_values('top_1_sim', ascending=False)
# could calculate here, but would be more complex and not needed

sorted_test_df['text_sim'] = np.nan


In [ ]:
sorted_test_df['text_equal'] = (sorted_test_df[COL_NAME_NAME] + sorted_test_df[COL_NAME_DESCRIPTION]) == (sorted_test_df[SIM_PREFIX + COL_NAME_NAME] + sorted_test_df[SIM_PREFIX + COL_NAME_DESCRIPTION])
sorted_test_df['name_equal'] = (sorted_test_df[COL_NAME_NAME]) == (sorted_test_df[SIM_PREFIX + COL_NAME_NAME])

In [ ]:
# whole histogram
sorted_test_df['top_1_sim'].plot.hist(bins=100)

In [ ]:
# zoom in
sorted_test_df[sorted_test_df['top_1_sim'] >= 0.97]['top_1_sim'].plot.hist(bins=100)


In [ ]:
def text_shortener(text: str):
    if len(text) <= 60:
        return text
    return text[:60] + "..."


def display_comparison(df: pd.DataFrame, min_sim: float, column='top_1_sim'):
    return display(
        HTML(
            df[df[column] > min_sim][-20:][[COL_NAME_IMAGE_FILE, 'sim_' + COL_NAME_IMAGE_FILE, 'top_1_sim', COL_NAME_NAME, SIM_PREFIX + COL_NAME_NAME, 'text_sim', 'text_equal', 'name_equal', COL_NAME_DESCRIPTION, SIM_PREFIX + COL_NAME_DESCRIPTION]].to_html(
                formatters={
                    COL_NAME_IMAGE_FILE: image_formatter,
                    'sim_' + COL_NAME_IMAGE_FILE: image_formatter,
                    COL_NAME_DESCRIPTION: text_shortener,
                    SIM_PREFIX + COL_NAME_DESCRIPTION: text_shortener,
                },
                escape=False,
                index=False,
            )
        )
    )


In [ ]:
display_comparison(sorted_test_df, 0.99)


In [ ]:
display_comparison(sorted_test_df, 0.98)


In [ ]:
display_comparison(sorted_test_df, 0.975)


In [ ]:
# nearly the same images
img_sim_limit = 0.99
nearly_leak_test_images_df: pd.DataFrame = sorted_test_df.loc[sorted_test_df['top_1_sim'] > img_sim_limit].copy()


In [ ]:
# the same texts
(test_df[COL_NAME_NAME] + test_df[COL_NAME_DESCRIPTION]).isin((train_df[COL_NAME_NAME] + train_df[COL_NAME_DESCRIPTION]).unique()).sum()

In [ ]:
# nearly the same image and the same text
(nearly_leak_test_images_df[COL_NAME_NAME] + nearly_leak_test_images_df[COL_NAME_DESCRIPTION]).isin((train_df[COL_NAME_NAME] + train_df[COL_NAME_DESCRIPTION]).unique()).sum()


In [ ]:
test_text_embeddings_array = load_embeddings(test_df, CLIP_TEXTUAL_EMBS_DIR)
train_text_embeddings_array = load_embeddings(train_df, CLIP_TEXTUAL_EMBS_DIR)


In [ ]:

nearly_leak_test_images_test_text_embs = test_text_embeddings_array[nearly_leak_test_images_df['index'].values]
nearly_leak_test_images_train_text_embs = train_text_embeddings_array[nearly_leak_test_images_df['top_1_train_index'].values]
assert nearly_leak_test_images_train_text_embs.shape == nearly_leak_test_images_test_text_embs.shape
nearly_leak_test_images_sims = [nearly_leak_test_images_test_text_embs[i] @ nearly_leak_test_images_train_text_embs[i].transpose() for i in range(nearly_leak_test_images_train_text_embs.shape[0])]
nearly_leak_test_images_df['text_sim'] = nearly_leak_test_images_sims



In [ ]:
nearly_leak_test_images_train_text_embs.shape

In [ ]:
nearly_leak_test_images_sims[:10]

In [ ]:
nearly_same_images_sorted_df = nearly_leak_test_images_df.sort_values('text_sim', ascending=False)
nearly_same_images_sorted_df['text_sim'].plot.hist(bins=100)


In [ ]:
nearly_same_images_sorted_df['text_sim'][nearly_same_images_sorted_df['text_sim'] > 0.95].plot.hist(bins=100)


In [ ]:
display_comparison(nearly_same_images_sorted_df, 0.99, 'text_sim')


In [ ]:
display_comparison(nearly_same_images_sorted_df, 0.98, 'text_sim')

In [ ]:
nearly_same_images_sorted_df[nearly_same_images_sorted_df['text_sim'] > 0.99].shape

In [ ]:
nearly_same_images_sorted_df[nearly_same_images_sorted_df['text_sim'] == 1].shape


In [ ]:
nearly_same_items_df = nearly_same_images_sorted_df[nearly_same_images_sorted_df['text_sim'] > 0.99]

In [ ]:
# image_id is not unique.
test_df[COL_NAME_IMAGE_ID].isin(nearly_same_items_df[COL_NAME_IMAGE_ID].tolist()).sum()

In [ ]:
# plus remove all indentical texts to increase symmetry between the modalities

In [ ]:
len(test_df)

In [ ]:
pd.Series([np.nan, 1]).isin([np.nan])

In [ ]:
mask1 = test_df['index'].isin(nearly_same_items_df['index'])
mask2 = (test_df[COL_NAME_NAME] + test_df[COL_NAME_DESCRIPTION]).isin((train_df[COL_NAME_NAME] + train_df[COL_NAME_DESCRIPTION]).unique())
print(mask1.sum())
print(mask2.sum())

no_sim_test_df = test_df[~(mask1) & ~(mask2)]
len(test_df) - len(no_sim_test_df)

In [ ]:
mask1 = test_df['index'].isin(nearly_same_items_df['index'])
mask2 = (test_df[COL_NAME_NAME] + test_df[COL_NAME_DESCRIPTION]).isin((train_df[COL_NAME_NAME] + train_df[COL_NAME_DESCRIPTION]).unique())
sim_only_test_df = test_df[mask1 | mask2]


In [ ]:
from _csv import QUOTE_NONNUMERIC


no_sim_test_df.to_csv(EXTRACT_DIR + 'GLAMI-1M-test.csv-no-sim-2022-10-04.csv', quoting=QUOTE_NONNUMERIC, index=False)
sim_only_test_df.to_csv(EXTRACT_DIR + 'GLAMI-1M-test.csv-sim-only-2022-10-04.csv')
no_sim_test_df.head()


In [ ]:
sim_only_test_df.head()